In [4]:
!cd ./Gemini-Hackathon

In [5]:
!pwd

/data


In [ ]:
# Installation of Python-3.11
conda install -n base -c conda-forge python=3.11 -y

In [ ]:
conda create -n skyhammer311 -c conda-forge python=3.11 -y
conda activate skyhammer311
which python

python -m pip install -U pip
pip install -r requirements.txt


huggingface-cli login
wandb login

In [ ]:
# 1. Create Environment
conda create -n skyhammer_serve -c conda-forge python=3.11 -y

# 2. Activate
conda activate skyhammer_serve
which python

# 3. Install vLLM (Clean install)
pip install -r requirements-serve.txt

In [ ]:
# Just press Enter for defaults, mostly. 
# Key choices: 
# - Multi-GPU: YES
# - Num machines: 1
# - Num processes: 8
# - Mixed precision: bf16 (Since you have L40s)
accelerate config

export PYTORCH_ALLOC_CONF=expandable_segments:True
accelerate launch --multi_gpu --num_processes=8 src/train/train_ddp.py

In [ ]:
# 1. Set Memory Management Flag     # 2. Run on Single GPU (Safe Mode)
export PYTORCH_ALLOC_CONF=expandable_segments:True
CUDA_VISIBLE_DEVICES=0 python src/train/train_ddp.py

In [ ]:
conda deactivate
conda remove --name skyhammer311 --all -y

# VLLM SERVE

In [ ]:
vllm serve Qwen/Qwen2.5-Coder-32B-Instruct-AWQ \
  --tensor-parallel-size 2 \
  --enable-lora \
  --lora-modules skyhammer=shng2025/SkyHammer-32B-v1 \
  --port 8000 \
  --api-key skyhammer

In [ ]:
vllm serve Qwen/Qwen3-VL-32B-Instruct \
  --tensor-parallel-size 2 \
  --max-model-len 32768 \
  --port 8000 \
  --trust-remote-code \
  --api-key skyhammer

In [ ]:
huggingface-cli download unsloth/GLM-4.7-GGUF \
  --include "Q4_K_M/*" \
  --local-dir models/ \
  --local-dir-use-symlinks False


# pip install "llama-cpp-python[server]"
CMAKE_ARGS="-DGGML_CUDA=on" pip install --upgrade --force-reinstall --no-cache-dir llama-cpp-python[server]

python -m llama_cpp.server \
  --model models/Q4_K_M/GLM-4.7-Q4_K_M-00001-of-00005.gguf \
  --n_gpu_layers -1 \
  --n_ctx 32768 \
  --port 8000 \
  --api_key skyhammer




In [ ]:
# Run inside 'skyhammer311' environment
accelerate launch --multi_gpu --num_processes=8 src/train/train_ddp-glm.py \
  --max_steps 10 \
  --logging_steps 1